In [1]:
from optparse import OptionParser
import os, sys, shutil, copy, os.path
sys.path.append(os.environ['SU2_RUN'])
import SU2
from SU2.io import redirect_folder, redirect_output

In [2]:
# Input options
config_fn = 'turb_flatplate.cfg'

# Config parsing
config = SU2.io.Config(config_fn)
state  = SU2.io.State()

# Location to copy meshes from:
mesh_loc = '/home/jmukho/archive/meshes/flatplate/su2/vandv'
mesh_files = [f for f in os.listdir(mesh_loc) if os.path.isfile(os.path.join(mesh_loc, f))]

# copy mesh files
print('Copying mesh files to root folder... ')
for f in mesh_files:
    shutil.copy(os.path.join(mesh_loc,f),'.')
print('Copying complete')

# folder set up
mesh_folders = ['L5','L4','L3','L2','L1']
models = ['SA', 'SST']
grads = {'gg' : 'GREEN_GAUSS', 'wlsq' : 'WEIGHTED_LEAST_SQUARES'}
recon_grads = ['ls','']
diverged_cases = []
num_cores = {'L5': 1, 'L4' : 1, 'L3' : 4, 'L2' : 18, 'L1' : 18}

for model in models:
    print('Running mesh convergence study for the ' + model + ' turbulence model: ')
    for grad_folder,grad in grads.items():
        print('\tUsing NUM_GRADIENT_METHOD = ' + grad)
        for recon in recon_grads:
            if recon:
                print('\t\tUsing NUM_GRADIENT_METHOD_RECON = LEAST_SQUARES')
            else:
                recon = '2nd_order'
                print('\t\tUsing second order reconstructions')
            for level in mesh_folders:
                curr_mesh = ''
                for mesh in mesh_files:
                    if level in mesh:
                        curr_mesh = mesh
                        break
                print('\t\t\tRunning ' + level + ' mesh level using ' + curr_mesh)
                run_folder = '{}/{}/{}/{}/'.format(model,grad_folder,recon,level)
                print('\t\t\t'+run_folder)
            
                # make copies
                konfig = copy.deepcopy(config)
                ztate  = copy.deepcopy(state)

                # update options
                konfig.KIND_TURB_MODEL= model
                konfig.NUM_GRADIENT_METHOD= grad
                if 'ls' in recon:
                    konfig.NUM_GRADIENT_METHOD_RECON= recon
                konfig.MESH_FILENAME= curr_mesh
                konfig.NUMBER_PART = num_cores[level]
                
                ztate.find_files(konfig)
                
                link = []
                link.append(ztate.FILES['MESH'])
                
                with redirect_folder(run_folder,[],link, force=False) as push:
                    with redirect_output('log.out'):
                        try:
                            print("running")
#                             info = SU2.run.CFD(konfig)
#                             ztate.update(info)
                        except:
                            print("The solver stopped unexpectedly")
                            diverged_cases.append(run_folder)
                            print("Continuing... ")
                
                print("Following cases diverged: ")
                print(diverged_cases)
            


Copying mesh files to root folder... 
Copying complete
Running mesh convergence study for the SA turbulence model: 
	Using NUM_GRADIENT_METHOD = GREEN_GAUSS
		Using NUM_GRADIENT_METHOD_RECON = LEAST_SQUARES
			Running L5 mesh level using flatplate_turb_L5_35x25.su2
			SA/gg/ls/L5/
Found: flatplate_turb_L5_35x25.su2
Following cases diverged: 
[]
			Running L4 mesh level using flatplate_turb_L4_69x49.su2
			SA/gg/ls/L4/
Found: flatplate_turb_L4_69x49.su2
Following cases diverged: 
[]
			Running L3 mesh level using flatplate_turb_L3_137x97.su2
			SA/gg/ls/L3/
Found: flatplate_turb_L3_137x97.su2
Following cases diverged: 
[]
			Running L2 mesh level using flatplate_turb_L2_273x193.su2
			SA/gg/ls/L2/
Found: flatplate_turb_L2_273x193.su2
Following cases diverged: 
[]
			Running L1 mesh level using flatplate_turb_L1_545x385.su2
			SA/gg/ls/L1/
Found: flatplate_turb_L1_545x385.su2
Following cases diverged: 
[]
		Using second order reconstructions
			Running L5 mesh level using flatplate_turb_

In [23]:
files = [f for f in os.listdir(os.getcwd()) if os.path.isfile(os.path.join(os.getcwd(), f))]
mesh_files = [f for f in files if '.su2' in f]
print(mesh_files)

['flatplate_turb_L1_545x385.su2', 'flatplate_turb_L2_273x193.su2', 'flatplate_turb_L3_137x97.su2', 'flatplate_turb_L4_69x49.su2', 'flatplate_turb_L5_35x25.su2']
